In [56]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [57]:
df_review=pd.read_csv('/kaggle/input/lazada-indonesian-reviews/20191002-reviews.csv')

In [58]:
import tensorflow as tf


In [59]:
import pandas as pd

In [60]:
df_review

In [61]:
df=df_review.loc[:,['rating','reviewContent']]

In [62]:
df=df.dropna().reset_index(drop=True)

In [63]:
df[df['reviewContent'].str.contains('\?')].index

In [64]:
df=df.drop(index=df[df['reviewContent'].str.contains('\?')].index).reset_index(drop=True)

In [65]:
import string

In [66]:
table=str.maketrans("","",string.punctuation)

In [67]:
def lowertext(text):
  return text.lower()
def remove_puncts(text):
    return text.translate(table)
def preprocessingtext(text):
  text=lowertext(text)
  text=remove_puncts(text)
  return(text)  

In [68]:
temp=df[df['reviewContent']=='...'].index

In [69]:
df=df.drop(index=temp).reset_index(drop=True)

In [70]:
df['reviewCleaned']=df['reviewContent'].apply(preprocessingtext)

In [93]:
maxlen=df['reviewCleaned'].str.split().str.len().max()
print(maxlen)

In [72]:
import numpy as np

In [73]:
X=df['rating'].values
X.shape

In [74]:
from sklearn.preprocessing import OneHotEncoder
# example of a one hot encoding
from numpy import asarray
oh=OneHotEncoder(sparse=False)
y=oh.fit_transform(X=X.reshape(-1,1))

In [75]:
y.shape

In [76]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(df['reviewCleaned'],y,train_size=0.3)

In [77]:
y_train.shape

In [78]:
df['rating'].value_counts()

In [85]:
tokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df["reviewCleaned"].values)

In [92]:
tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [86]:
x_train_sequence=tokenizer.texts_to_sequences(x_train.values)
x_test_sequence=tokenizer.texts_to_sequences(x_test.values)

In [87]:
X_train_sequence =np.array(tf.keras.preprocessing.sequence.pad_sequences(x_train_sequence, maxlen=maxlen, padding="post"))
X_test_sequence = np.array(tf.keras.preprocessing.sequence.pad_sequences(x_test_sequence, maxlen=maxlen, padding="post"))


In [88]:
X_train_sequence

## Model Deep Learning

In [27]:
model=tf.keras.Sequential()

In [28]:
X_train_sequence

In [29]:
inputs=tf.keras.layers.Input(shape=(maxlen,))
embed=tf.keras.layers.Embedding(10000,32,input_length=maxlen)(inputs)
gru=tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32))(embed)
layer1=tf.keras.layers.Dense(64,activation='relu')(gru)
output=tf.keras.layers.Dense(5,activation='softmax')(layer1)
model_lstm = tf.keras.models.Model(inputs=inputs, outputs=output)

# 3. Compile
model_lstm.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)

# 4. Cek struktur model
model_lstm.summary()

In [30]:
history=model_lstm.fit(x=X_train_sequence,y=y_train,epochs=25,batch_size=32)

In [31]:
# Evaluasi
loss, acc = model_lstm.evaluate(X_test_sequence, y_test)
print(f"Model test loss: {loss}")
print(f"Model test acc: {acc}")

In [35]:
model_lstm.save('model_gru', save_format="h5")

In [32]:
print(tokenizer_dict)


In [33]:
import math 
def predict(texts, model):
    texts = [preprocessingtext(text) for text in texts]
    seq = tokenizer.texts_to_sequences(texts)
    seq = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=maxlen, padding="post")
    probs = model.predict(seq)
    return probs
samples = ['seller ramah',
           'pengiriman cepat',
            'packaging buruk sehingga barang mudah rusak']

probs = predict(samples, model_lstm)
for sample, prob in zip(samples, probs):
    print(f"Kalimat: {sample}")
    print(f"Probabilitas: {(prob)}")
    print()



In [ ]:
math.e

In [ ]:
probs

In [ ]:

for sample, prob in zip(samples, probs):
    print(f"Kalimat: {sample}")
    print(f"Probabilitas: {prob}")
    print()